In [13]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO
from tqdm import tqdm
import warnings

In [19]:
dir_path = '/opt/ml/code/mmdetection_trash/Mybaseline/ensemble/final'

submission_files = ['(config11)kfold_ensemble_512_wbf_ths0.0.csv',
                    'sabl_faster_resnest50_pafpn_kfold.csv',
                    'sm25_sm20+testMS[384,512,1024].csv',
                    'swin_b_tta_36.csv']

weights = [5.2**3,
          5.232**3,
          4.943**3,
          4.946**3]

#                     ['(config11)fold0_test_512.csv',
#                     '(config11)fold0_flip_512.csv',
#                     '(config11)fold0_rotate_512.csv',
#                     '(config11)fold0_rotateR_512.csv'] + \
#                     ['(config11)fold1_test_512.csv',
#                     '(config11)fold1_flip_512.csv',
#                     '(config11)fold1_rotate_512.csv',
#                     '(config11)fold1_rotateR_512.csv'] + \
#                     ['(config11)fold2_test_512.csv',
#                     '(config11)fold2_flip_512.csv',
#                     '(config11)fold2_rotate_512.csv',
#                     '(config11)fold2_rotateR_512.csv'] + \
#                     ['(config11)fold3_test_512.csv',
#                     '(config11)fold3_flip_512.csv',
#                     '(config11)fold3_rotate_512.csv',
#                     '(config11)fold3_rotateR_512.csv'] + \
#                     ['(config11)fold4_test_512.csv',
#                     '(config11)fold4_flip_512.csv',
#                     '(config11)fold4_rotate_512.csv',
#                     '(config11)fold4_rotateR_512.csv']# + \
#                     ['(config11)fold0_test_1024.csv',
#                     '(config11)fold0_flip_1024.csv',
#                     '(config11)fold0_rotate_1024.csv',
#                     '(config11)fold0_rotateR_1024.csv'] + \
#                     ['(config11)fold1_test_1024.csv',
#                     '(config11)fold1_flip_1024.csv',
#                     '(config11)fold1_rotate_1024.csv',
#                     '(config11)fold1_rotateR_1024.csv'] + \
#                     ['(config11)fold2_test_1024.csv',
#                     '(config11)fold2_flip_1024.csv',
#                     '(config11)fold2_rotate_1024.csv',
#                     '(config11)fold2_rotateR_1024.csv'] + \
#                     ['(config11)fold3_test_1024.csv',
#                     '(config11)fold3_flip_1024.csv',
#                     '(config11)fold3_rotate_1024.csv',
#                     '(config11)fold3_rotateR_1024.csv'] + \
#                     ['(config11)fold4_test_1024.csv',
#                     '(config11)fold4_flip_1024.csv',
#                     '(config11)fold4_rotate_1024.csv',
#                     '(config11)fold4_rotateR_1024.csv']
submission_df = [pd.read_csv(f'{dir_path}/{file}') for file in submission_files]

In [20]:
image_ids = submission_df[0]['image_id'].tolist()

In [21]:
annotation = '/opt/ml/input/data/test.json'
coco = COCO(annotation)
warnings.filterwarnings(action='ignore')

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [22]:
prediction_strings = []
file_names = []
iou_thr = 0.4

for i, image_id in tqdm(enumerate(image_ids)):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        if len(predict_list)==0 or len(predict_list)==1:
            continue
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights)#nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            if score > 0.0:
                prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

837it [00:25, 33.42it/s]


In [23]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('/opt/ml/code/mmdetection_trash/Mybaseline/ensemble/final/ffinal_iouthr0.55_withoutyolo_3time.csv', index=False)

submission.head()

,PredictionString,image_id
0,8 0.9962050914764404 170.8434600830078 234.363...,batch_01_vt/0021.jpg
1,8 0.9757473468780518 156.59963989257812 63.001...,batch_01_vt/0028.jpg
2,1 0.9128338694572449 74.40225982666016 167.755...,batch_01_vt/0031.jpg
3,8 1.2444509267807007 66.1553726196289 147.9190...,batch_01_vt/0032.jpg
4,8 0.9991155862808228 150.00592041015625 0.1535...,batch_01_vt/0070.jpg
